In [41]:
print("OK!")

OK!


In [42]:
from langchain import PromptTemplate 
from langchain.chains import RetrievalQA 
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

In [43]:
PINECONE_API_KEY = "7eeff629-f281-4e06-a11b-eae034db2e00"
PINECONE_API_ENV = "gcp-starter"

In [44]:
#EXTRACT DATA FROM THE PDF 
def load_pdf(data): 
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load() 

    return documents
# import PyPDF2
# def load_pdf(data):
#     with open(data, 'rb') as f:
#         pdf_reader = PyPDF2.PdfFileReader(f)
#         text = ''
#         for page in pdf_reader.pages:
#             text += page.extractText()
#     return text


In [45]:
extracted_data = load_pdf("data/")

In [46]:
def text_splitter(extracted_data): 
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,chunk_overlap=20) 
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [47]:
text_chunk = text_splitter(extracted_data) 
print(text_chunk[0])
print(len(text_chunk))

page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION' metadata={'source': 'data\\Medical_book.pdf', 'page': 1}
7020


In [48]:
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") 
    return embeddings


In [49]:
embeddings = download_hugging_face_embedding()

In [50]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [51]:
query_result = embeddings.embed_query("Hello World") 

print(len(query_result))
print(query_result)


384
[-0.034477267414331436, 0.031023206189274788, 0.006734929047524929, 0.026108982041478157, -0.03936203196644783, -0.16030243039131165, 0.06692398339509964, -0.006441489793360233, -0.04745049402117729, 0.014758839271962643, 0.07087527960538864, 0.05552763119339943, 0.01919335499405861, -0.026251323521137238, -0.010109569877386093, -0.02694045566022396, 0.022307392209768295, -0.02222662791609764, -0.14969263970851898, -0.017493031919002533, 0.007676294539123774, 0.054352253675460815, 0.0032544503919780254, 0.031725917011499405, -0.08462144434452057, -0.029405998066067696, 0.051595620810985565, 0.048124048858881, -0.00331486901268363, -0.058279212564229965, 0.041969265788793564, 0.022210726514458656, 0.12818878889083862, -0.02233896590769291, -0.011656217277050018, 0.06292834132909775, -0.032876402139663696, -0.09122605621814728, -0.03117532841861248, 0.05269954353570938, 0.047034818679094315, -0.08420311659574509, -0.030056139454245567, -0.020744847133755684, 0.00951785035431385, -0.0

In [52]:

# # pc  = api_key=PINECONE_API_KEY,environment=PINECONE_API_ENV
# # import pinecone
# # from pinecone import Pinecone
# from langchain_pinecone import PineconeVectorStore
# pc = Pinecone(api_key = PINECONE_API_KEY)

# index_name = "medical-chatbot"
# # docsearch=Pinecone.from_texts([t.page_content for t in text_chunk] , embeddings , index_name = index_name)
# index = pc.Index(index_name)  
# for i, t in zip(range(len(text_chunk)), text_chunk):
#    query_result = embeddings.embed_query(t.page_content)
#    index.upsert(
#    vectors=[
#         {
#             "id": str(i),  # Convert i to a string
#             "values": query_result, 
#             "metadata": {"text":str(text_chunk[i].page_content)} # meta data as dic
#         }
#     ],
#     namespace="real" 
# )

# index.describe_index_stats() 



In [53]:
%pip install -U langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


In [59]:
# import PyPDF2
# # from pinecone import Pinecone
# from langchain_pinecone import PineconeVectorStore
# index = PineconeVectorStore.Index("medical-chatbot")
# PineconeVectorStore
# # docs = pc.similarity_search(query,k=3) 
# print("Result" , docs) 
from langchain_pinecone import PineconeVectorStore
import os 
os.environ['PINECONE_API_KEY'] = "7eeff629-f281-4e06-a11b-eae034db2e00"
api_key = os.environ["PINECONE_API_KEY"]
index_name = "medical-chatbot"

docsearch = PineconeVectorStore.from_documents(
    text_chunk,
    embeddings,
    index_name = index_name
)
query = "What is allergy?"

docs = docsearch.similarity_search(query)
print(docs)





[Document(page_content='When thisoccurs, an allergy develops against the offending sub-stance (an allergen.)', metadata={'page': 128.0, 'source': 'data\\Medical_book.pdf'}), Document(page_content='When thisoccurs, an allergy develops against the offending sub-stance (an allergen.)', metadata={'page': 128.0, 'source': 'data\\Medical_book.pdf'}), Document(page_content='When thisoccurs, an allergy develops against the offending sub-stance (an allergen.)', metadata={'page': 128.0, 'source': 'data\\Medical_book.pdf'}), Document(page_content='the itchy, scratchy nose, eyes, and throat common inallergic rhinitis .\nThe particular allergens to which a person is sensi-', metadata={'page': 135.0, 'source': 'data\\Medical_book.pdf'})]
